In [ ]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

# walk between 2 sentences in the latent space of a pretrained GPT-2 model

# Load GPT-2 with the language modeling head
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define two input sentences
sentence1 = "A peaceful village rests by the lake."
sentence2 = "A futuristic city glows under neon lights."

# Convert sentences into embeddings
tokens1 = tokenizer(sentence1, return_tensors="pt")["input_ids"]
tokens2 = tokenizer(sentence2, return_tensors="pt")["input_ids"]

# Get word embeddings
with torch.no_grad():
    embedding1 = model.get_input_embeddings()(tokens1).squeeze(0)
    embedding2 = model.get_input_embeddings()(tokens2).squeeze(0)

# Ensure embeddings have the same length (pad if necessary)
max_len = max(embedding1.shape[0], embedding2.shape[0])
embedding1 = torch.nn.functional.pad(embedding1, (0, 0, 0, max_len - embedding1.shape[0]))
embedding2 = torch.nn.functional.pad(embedding2, (0, 0, 0, max_len - embedding2.shape[0]))

# Define SLERP function (Spherical Linear Interpolation)
# change num steps 
def slerp(v0, v1, num_steps=10):
    v0, v1 = v0.numpy(), v1.numpy()
    dot = np.sum(v0 * v1, axis=-1) / (np.linalg.norm(v0, axis=-1) * np.linalg.norm(v1, axis=-1))
    dot = np.clip(dot, -1.0, 1.0)
    theta = np.arccos(dot)
    sin_theta = np.sin(theta)

    interpolated_vectors = []
    for t in np.linspace(0, 1, num_steps):
        v = (np.sin((1 - t) * theta) / sin_theta)[:, None] * v0 + (np.sin(t * theta) / sin_theta)[:, None] * v1
        interpolated_vectors.append(torch.tensor(v, dtype=torch.float32))

    return interpolated_vectors

# Generate interpolated latents
num_steps = 30
interpolated_latents = slerp(embedding1, embedding2, num_steps=num_steps)

# Decode the interpolated embeddings into words
decoded_sentences = []
for i, latent in enumerate(interpolated_latents):
    with torch.no_grad():
        token_logits = model.lm_head(latent)  # ✅ Now works with `AutoModelForCausalLM`
        token_ids = torch.argmax(token_logits, dim=-1)
        decoded_text = tokenizer.decode(token_ids, skip_special_tokens=True)
    
    decoded_sentences.append(decoded_text)
    print(f"Step {i}: {decoded_text}")


Step 0: ! peaceful village rests by the lake.!
Step 1: ! peaceful village rests by the lake.!
Step 2: ! peaceful village rests by the lake.!
Step 3: ! peaceful village rests by the lake.!
Step 4: ! peaceful village rests by the lake.!
Step 5: ! peaceful village rests by the lake.!
Step 6: ! peaceful village restsows the lake.!
Step 7: ! peaceful village restsows the lake.!
Step 8: ! peaceful village restsows the lake.!
Step 9: ! peaceful village restsows the lake.!
Step 10: ! peaceful village restsows the lake.!
Step 11: ! peaceful village restsows the lake.!
Step 12: ! peaceful village restsows the lake lights!
Step 13: ! peaceful village restsows the lake lights!
Step 14: ! peaceful village restsows under neon lights!
Step 15: ! futuristic village restsows under neon lights!
Step 16: ! futuristic village restsows under neon lights!
Step 17: ! futuristic village restsows under neon lights!
Step 18: ! futuristic city restsows under neon lights!
Step 19: ! futuristic city glows under ne

/tmp/ipykernel_52636/3160894687.py:33: RuntimeWarning: invalid value encountered in divide
  dot = np.sum(v0 * v1, axis=-1) / (np.linalg.norm(v0, axis=-1) * np.linalg.norm(v1, axis=-1))
/tmp/ipykernel_52636/3160894687.py:40: RuntimeWarning: invalid value encountered in divide
  v = (np.sin((1 - t) * theta) / sin_theta)[:, None] * v0 + (np.sin(t * theta) / sin_theta)[:, None] * v1


In [ ]:
# Get answer to a question and walk around the answer randomly